tochvision 主要处理图像数据，包含一些常用的数据集、模型、转换函数等。torchvision独立于PyTorch，需要专门安装，torchvision主要包含以下四部分：

torchvision.models: 提供深度学习中各种经典的网络结构、预训练好的模型，如：Alex-Net、VGG、ResNet、Inception等

torchvision.datasets：提供常用的数据集，设计上继承 torch.utils.data.Dataset，主要包括：MNIST、CIFAR10/100、ImageNet、COCO等

torchvision.transforms：提供常用的数据预处理操作，主要包括对Tensor及PIL Image对象的操作

torchvision.utils：工具类，如保存张量作为图像到磁盘，给一个小批量创建一个图像网格

torchvision要注意与pytorch版本和Cuda相匹配。要查询pytorch和torchvision的版本，可以使用下面语句：

In [10]:
import torch
import torchvision
print(torch.__version__)
print(torchvision.__version__)

2.8.0+cu126
0.23.0+cu126


(1) 加载几个预训练模型

通过 pretrined=True 可以加载预训练模型。pretrained 默认值是 False，不赋值和赋值 False 效果一样

选择了 pretrained 为 True 的话，点击运行会执行下载预训练参数的操作，不加载预训练模型没有这步操作，可以直接使用网络架构

In [ ]:
import torchvision.models as models
from torchsummary import summary as sumy

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

resnet18 = models.resnet18(weights='IMAGENET1K_V1')
vgg16 = models.vgg16(weights='IMAGENET1K_V1')
alexnet = models.alexnet(weights='IMAGENET1K_V1')
squeezenet = models.squeezenet1_0(weights='IMAGENET1K_V1')
densenet = models.densenet161(weights='IMAGENET1K_V1')
resnet152 = models.resnet152()
# 可以使用该方法打印加载的预训练模型的参数
for name, param in vgg16.named_parameters():
    print(name, param.shape)
vgg16 = vgg16.to(device)
sumy(vgg16, input_size=(3, 224, 224), device=str(device))


<bound method Module.named_parameters of VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kerne

vgg16.state_dict() 前面方法得到模型的参数

加载本地的模型参数使用方法 vgg16.load_state_dict()

需要结合加载文件转化为字典的函数 torch.load_dict() 一起使用，也就是 vgg16.load_state_dict(torch.load_dict(xxxx.pth))

保存模型的参数需要得到模型和的字典，然后使用 torch.save() 函数来保存，torch.save(vgg16.state_dict(), "xxxx.pth")

In [12]:
vgg16_state_dict = vgg16.state_dict()
resnet18_state_dict = resnet18.state_dict()

torch.save(vgg16_state_dict, "./vgg16_state_dict.pth")
torch.save(resnet18_state_dict, "./resnet18_state_dict.pth")

vgg16.load_state_dict(torch.load("./vgg16_state_dict.pth"))

<All keys matched successfully>

但是正常的使用过程中可能会修改模型的层数，想要接着使用预训练的参数就会出现无法加载的问题，所以必须在 pth 文件里面删掉或者加上相应的修改的层

加入现在 resnet18 的分类任务变为了分类为 10 类，就需要对最后的全连接层进行修改

层的名字和 shape 都是匹配的话，成功的加载，名字不匹配但是形状匹配的话，strict=False 会忽略错误，名字匹配但是形状不匹配，都会报错

pretrained_dict = {k : v for k, v in pretrained_dict.items() if k in model_dict} 这个写法很显然只会比较键是不是完全一样，即使修改了形状也不会关心，所以需要加上判断，v.shape 和 model_dict[k].shape 要是一样的才行，这样的话，只会从预训练模型里面加载确定都没有修改的层

In [ ]:
import torch.nn as nn

model = models.resnet18(weights=None)

model.fc = nn.Linear(512, 10)
pretrained_dict = resnet18.state_dict()
model_dict = model.state_dict()
print(pretrained_dict.keys() == model_dict.keys())
print(pretrained_dict['fc.bias'].size())
pretrained_dict = {k : v for k, v in pretrained_dict.items() if k in model_dict and v.shape == model_dict[k].shape}
# print(pretrained_dict['fc.bias']) 这句话会报错因为修改了全连接层，这个参数没有被加载
# res = model.load_state_dict(torch.load('./resnet18_state_dict.pth'), strict=False) 报错，因为无法对应，原本的预训练模型的全连接层的大小为 [512, 1000]，现在被我修改为了 [512, 10]
res = model.load_state_dict(pretrained_dict, strict=False) # 完美的加载，验证了自己的想法，现在删掉了全连接层的参数，其他的参数的预训练的参数了

print("Missing keys:", res.missing_keys) # 模型中存在，但是在预训练参数里面找不到
print("Unexpected keys:", res.unexpected_keys) # 预训练参数中存在模型中没定义，这个不会遇到，因为我强制的比对了键值对，模型中没定义的一定会被删掉的

print(model.fc.weight.mean(), model.fc.weight.std())  # 随机分布
print(resnet18.fc.weight.mean(), resnet18.fc.weight.std())  # 不同分布


True
torch.Size([1000])
Missing keys: ['fc.weight', 'fc.bias']
Unexpected keys: []
tensor(9.4485e-06, grad_fn=<MeanBackward0>) tensor(0.0254, grad_fn=<StdBackward0>)
tensor(5.8502e-08, grad_fn=<MeanBackward0>) tensor(0.0695, grad_fn=<StdBackward0>)


上面简单的介绍了模型的加载和报错，和加载预训练参数的方法等，这个是 torchvision.models 库函数里面常见的操作了，总结下列函数：

models.model() 加载名字为 model 的模型，一般的参数为 weights，代表加载预训练参数，不想加载的话初始化为 weights=None

model.named_parameters() 返回键值对，代表层的名字和对应的 shape


